In [11]:
!gdown --id 1wQ_CrnMkqCcBvw4bAi2NGVsHgt3cbE6G

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1wQ_CrnMkqCcBvw4bAi2NGVsHgt3cbE6G
From (redirected): https://drive.google.com/uc?id=1wQ_CrnMkqCcBvw4bAi2NGVsHgt3cbE6G&confirm=t&uuid=a257d7ec-a5cc-47c1-af3e-49bd0b22cb83
To: /kaggle/working/archive.zip
100%|████████████████████████████████████████| 421M/421M [00:09<00:00, 44.4MB/s]


In [15]:
import os
import shutil

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split, ConcatDataset
import timm
from torch import nn, optim
from torchvision.transforms import RandomResizedCrop, RandomHorizontalFlip, ColorJitter, Normalize
import pandas as pd
from PIL import Image

In [19]:
import os
import shutil
import pandas as pd

train_folder = '/kaggle/working/Fundus_Scanes_Sorted/Train'
val_folder = '/kaggle/working/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation'
combined_folder = '/kaggle/working/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Combined'
csv_file = '/kaggle/working/glaucoma.csv'

df = pd.read_csv(csv_file)

os.makedirs(combined_folder, exist_ok=True)

image_data = []

def add_images_to_dataframe(source_folder):
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)

                file_name = os.path.splitext(file)[0]

                label_row = df[df['Filename'] == file]
                if not label_row.empty:
                    glaucoma_label = label_row['Glaucoma'].values[0]
                    image_data.append({
                        'image_path': image_path,
                        'glaucoma': glaucoma_label
                    })
                else:
                    print(f"Warning: No label found for {file_name}, skipping.")

add_images_to_dataframe(os.path.join(train_folder, 'Glaucoma_Positive'))
add_images_to_dataframe(os.path.join(train_folder, 'Glaucoma_Negative'))
add_images_to_dataframe(os.path.join(val_folder, 'Glaucoma_Positive'))
add_images_to_dataframe(os.path.join(val_folder, 'Glaucoma_Negative'))

image_df = pd.DataFrame(image_data)

image_df.to_csv('/kaggle/working/combined_image_data.csv', index=False)

print(f"Total images processed: {len(image_df)}")
print("Image paths and labels are stored in the DataFrame.")

print(image_df.head())


Total images processed: 130
Image paths and labels are stored in the DataFrame.
                                          image_path  glaucoma
0  /kaggle/working/Fundus_Train_Val_Data/Fundus_S...         1
1  /kaggle/working/Fundus_Train_Val_Data/Fundus_S...         1
2  /kaggle/working/Fundus_Train_Val_Data/Fundus_S...         1
3  /kaggle/working/Fundus_Train_Val_Data/Fundus_S...         1
4  /kaggle/working/Fundus_Train_Val_Data/Fundus_S...         1


In [20]:
class GlaucomaDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = image_df
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
    image = Image.open(img_path).convert('RGB')
    glaucoma = torch.tensor(int(self.annotations.iloc[index, 1]))

    if self.transform:
      image = self.transform(image)

    return (image, glaucoma)

In [21]:
IMG_SIZE=224
transforms_all = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomRotation(degrees=45),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [22]:
dataset = GlaucomaDataset(csv_file=image_df, root_dir='/kaggle/working/', transform=transforms_all)


In [23]:
augmentation_factor = 13
augmented_dataset = ConcatDataset([dataset] * augmentation_factor)

In [24]:
train_size = int(0.8 * len(augmented_dataset))
val_size = len(augmented_dataset) - train_size

In [39]:
train_set, val_set = torch.utils.data.random_split(augmented_dataset, [train_size, val_size])
train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=32, shuffle=True)

In [40]:
model = timm.create_model("vit_base_resnet50d_224", pretrained=False)
model.classifier = nn.Linear(in_features=2816, out_features=2, bias=True)
print(model)

VisionTransformer(
  (patch_embed): HybridEmbed(
    (backbone): FeatureListNet(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [42]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [43]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        corrects += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = corrects / total

    return epoch_loss, epoch_acc

In [44]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    corrects = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            corrects += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = corrects / total

    return epoch_loss, epoch_acc

In [49]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    # Training
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')

    # Validation
    val_loss, val_acc = validate_model(model, val_loader, criterion, device)
    print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

Epoch 1/10
Train Loss: 0.2199 Acc: 0.9105
Val Loss: 0.1879 Acc: 0.9172
Epoch 2/10
Train Loss: 0.2395 Acc: 0.8935
Val Loss: 0.1846 Acc: 0.9260
Epoch 3/10
Train Loss: 0.2698 Acc: 0.8883
Val Loss: 0.2318 Acc: 0.8994
Epoch 4/10
Train Loss: 0.1866 Acc: 0.9231
Val Loss: 0.1611 Acc: 0.9260
Epoch 5/10
Train Loss: 0.2625 Acc: 0.9001
Val Loss: 0.1978 Acc: 0.9112
Epoch 6/10
Train Loss: 0.2555 Acc: 0.8987
Val Loss: 0.2007 Acc: 0.9320
Epoch 7/10
Train Loss: 0.1934 Acc: 0.9260
Val Loss: 0.1588 Acc: 0.9379
Epoch 8/10
Train Loss: 0.2122 Acc: 0.9075
Val Loss: 0.1602 Acc: 0.9438
Epoch 9/10
Train Loss: 0.2049 Acc: 0.9186
Val Loss: 0.1952 Acc: 0.9320
Epoch 10/10
Train Loss: 0.2228 Acc: 0.9157
Val Loss: 0.1490 Acc: 0.9438


In [50]:
PATH='/kaggle/working/vitmodel2.pth'

In [51]:
torch.save(model.state_dict(), PATH)

In [35]:
model = timm.create_model("vit_base_resnet50d_224", pretrained=False)
model.classifier = nn.Linear(in_features=2816, out_features=2, bias=True)
model.load_state_dict(torch.load(PATH))
model.eval()

/tmp/ipykernel_36/2812852892.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH))


VisionTransformer(
  (patch_embed): HybridEmbed(
    (backbone): FeatureListNet(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          